In [18]:
import sqlite3 as mydb
db = mydb.connect("python_database.db")
# cmd = "create table bank(acc_num int primary key ,acc_name varchar(300) ,acc_pin int , acc_balance decimal)"
# cmd2 = "create table bank_trans(acc_id int ,tr_amount real ,tr_type varchar(10) ,tr_date datetime)"
cursor = db.cursor()
# cursor.execute(cmd2)
db.commit()
# db.close()

In [60]:
import datetime
try:

    def verify_account(acc_num,acc_pin):
        query = """select * from bank where acc_num = ? and acc_pin = ?;"""
        cursor.execute(query,(acc_num,acc_pin,))
        data = cursor.fetchall()
        db.commit()
        if len(data) == 1:
            return 1
        else:
            return 0

    def create_account(acc_num,acc_name,acc_pin,acc_balance,acc_trans):
        
        data_tuple = (acc_num,acc_name,acc_pin,acc_balance,acc_trans)
        query = """insert into bank(acc_num, acc_name, acc_pin, acc_balance, acc_trans) values(?,?,?,?,?);"""
        result = cursor.execute(query, data_tuple)
        db.commit()
        if result:
            print("Account is created.")


    def acc_info(acc_num):
        cursor.execute("""select * from bank where acc_num = ?""",(acc_num,))
        data = cursor.fetchall()
        
        decor = ["acc_num","acc_name","acc_pin","acc_balance"]
        decor = [i.ljust(13) for i in decor]
        # decor_2 = "_________________________________________________________"
        
        # print(decor_2)
        # print(*decor)
        
        for item in data:
            for i in range(0,len(item)-1):
                print(decor[i],"=",item[i])            
        db.commit()
        
    def credit_money(acc_num):
        
        print(" Credit money ".center(60, "-"), end="\n\n")

        amount = input("Enter amount (without decimal value) : ")
        assert amount.isnumeric(), "Wrong input"
        amount = float(amount)
        
        # data = {
        #     "amount" : amount,
        #     "tr_type" : 'cr',
        #     "datetime": datetime.datetime.now()
        # }
        
        query = """insert into bank_trans(acc_id, tr_amount, tr_type, tr_date) values(?,?,?,?);"""
        result = cursor.execute(query,(acc_num,amount,"cr",datetime.datetime.now(),))

        if result: 
            cursor.execute("update bank set acc_balance = acc_balance + ? where acc_num = ?",(amount,acc_num,))

        print(f"{amount} credited into your account.")
        
        cursor.execute("""select acc_balance from bank where acc_num = ?""",(acc_num,))
        data = cursor.fetchall()
        for item in data:
            for i in item:
                acc_balance = i
        
        db.commit()
        print(f"Account balance : {acc_balance}")


    def debit_money(acc_num):
        cursor.execute("""select acc_balance from bank where acc_num = ?""", (acc_num,))
        data = cursor.fetchall()
        for item in data:
            for i in item:
                acc_balance = i
        
        amount = input("Enter amount (without decimal value) : ")
        assert amount.isnumeric(), "Wrong input"
        amount = float(amount)
        
        assert acc_balance >= amount, "No sufficient balance."
        
        print(" Debit money ".center(60, "-"), end="\n\n")    
        query = """insert into bank_trans(acc_id, tr_amount, tr_type, tr_date) values(?,?,?,?);"""
        result = cursor.execute(query, (acc_num, amount, "db", datetime.datetime.now(),))

        if result:
            cursor.execute("update bank set acc_balance = acc_balance - ? where acc_num = ?", (amount, acc_num,))
        db.commit()
        
        print(f"{amount} debited from your account.")
        print(f"Account balance : {acc_balance-amount}")

            
    def all_transactions(acc_num):

        cursor.execute("""select * from bank_trans where acc_id = ?""",(acc_num,))
        data = cursor.fetchall()

        decorate = [" All transactions ".center(60, "-") + "\n\n", "".center(60, "_") + "\n", "amount".center(15) + "cr/db".center(15) + "date & time".center(25)+ "\n", "".center(60, "_")]
        print(*decorate)
            
        for trans in data:
            for tr_items in range(1,len(trans)):
                print(str(trans[tr_items]).center(15), end="")
            print("\n")
            
    
except Exception as err:
    print(err)

In [61]:
try:
        
    def services(acc_num):

        while True:
            print("")
            print("1 -> Account Information")
            print("2 -> Credit money")
            print("3 -> Debit money")
            print("4 -> Transactions")
            print("5 -> Exit")

            choice = input("\nEnter number to access service :")
            assert choice.isnumeric(), "Wrong input."
            choice = int(choice)

            print("")

            if choice == 1:
                acc_info(acc_num)

            elif choice == 2:
                credit_money(acc_num)

            elif choice == 3:
                debit_money(acc_num)

            elif choice == 4:
                all_transactions(acc_num)

            elif choice == 5:

                print("Thanks for contact us".center(60, "_"))
                raise Exception("")

            else:
                print("Wrong choice.")


    print('Press "1" for create new account\nPress "2" for open existing account.')
    ans = input("Enter 1 or 2 : ")
    assert ans.isnumeric(), 'Wrong input.'
    ans = int(ans)

    if ans == 1:
        
        cursor.execute("select acc_num from bank")
        data = cursor.fetchall()
        # print(data)
        for index,item in enumerate(data):
            for num in item:
                last_acc_num = num
        
        if len(data) == 0:
            acc_num = 1001
        else:
            acc_num = last_acc_num + 1
        
        print(f"Your account number : {acc_num}")
        
        acc_name = input('Enter your name : ')
        acc_pin = input("Enter account pin : ")
        
        while not acc_pin.isnumeric() or not len(acc_pin) == 4:
            print("\nPlease enter four numbers only.")
            acc_pin = input("Enter account pin : ")

        acc_pin = int(acc_pin)
        
        create_account(acc_num,acc_name,acc_pin,0.00,"")
        services(acc_num)


    elif ans == 2:

        acc_num = input("Enter account number : ")
        acc_pin = input("Enter account pin : ")
        assert acc_pin.isnumeric(), "Wrong input."
        acc_num = int(acc_num)
        acc_pin = int(acc_pin)
        if (verify_account(acc_num, acc_pin)):
            services(acc_num)
        else:
            print("Invalid details.")

    else:
        print("Enter 1 or 2 only numbers.")

except Exception as err:
    print(err)

Press "1" for create new account
Press "2" for open existing account.

1 -> Account Information
2 -> Credit money
3 -> Debit money
4 -> Transactions
5 -> Exit

--------------------- All transactions ---------------------

 ____________________________________________________________
      amount         cr/db            date & time       
 ____________________________________________________________
     12345            cr      2022-07-09 13:32:55.229121

     12000            db      2022-07-09 14:11:12.589225

      1000            cr      2022-07-09 14:11:39.315387

      345             db      2022-07-09 14:11:49.877587


1 -> Account Information
2 -> Credit money
3 -> Debit money
4 -> Transactions
5 -> Exit

acc_num       = 1001
acc_name      = Himal
acc_pin       = 1212

1 -> Account Information
2 -> Credit money
3 -> Debit money
4 -> Transactions
5 -> Exit

___________________Thanks for contact us____________________



In [30]:
db.commit()